<a href="https://colab.research.google.com/github/trashchenkov/LLM-First-Touch/blob/main/LangMem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph langchain langchain-community langchain-gigachat langmem

In [ ]:
!pip install langchain-huggingface  langchain-groq

In [ ]:
from langchain.embeddings.base import _get_provider_list

# Вывод списка доступных провайдеров
print(_get_provider_list())


In [ ]:
from langchain_gigachat.chat_models import GigaChat
from langgraph.func import entrypoint
from langgraph.store.memory import InMemoryStore
from langmem import create_memory_store_manager
from langchain.embeddings import init_embeddings
from google.colab import userdata
from langchain_groq import ChatGroq

gc_key = userdata.get('GIGA')
groq_key = userdata.get('GROQ')

store = InMemoryStore( #
    index={
        "dims": 1024,
        "embed": "huggingface:intfloat/multilingual-e5-large-instruct",
    }
)
llm = GigaChat(
    credentials=gc_key, model='GigaChat-Max',
    verify_ssl_certs=False,
    scope="GIGACHAT_API_CORP"
)

llama = ChatGroq(
    model='llama-3.3-70b-versatile',
    api_key=groq_key
)


# Create memory manager Runnable to extract memories from conversations
memory_manager = create_memory_store_manager(
    llama,
    # Store memories in the "memories" namespace (aka directory)
    namespace=("memories",),  #
)

@entrypoint(store=store)  # Create a LangGraph workflow
async def chat(message: str):
    response = llm.invoke(message)

    # memory_manager extracts memories from conversation history
    # We'll provide it in OpenAI's message format
    to_process = {"messages": [{"role": "user", "content": message}] + [response]}
    await memory_manager.ainvoke(to_process)  #
    return response.content
# Run conversation as normal
response = await chat.ainvoke(
    "Я люблю кошек. Раньше у меня была кошка по кличке Шанти.",
)
print(response)


In [ ]:
print(store.search(("memories",)))

In [ ]:
for mem in store.search(("memories",)):
    print(mem)